In [23]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine, text

engine = create_engine('sqlite:///data.db', echo=False)
tables = ["DEPARTMENT", "INSTRUCTOR", "COURSE", "STUDY_GROUP", "ANNOUNCEMENT",  "CONTACT",
          "JOIN_GROUP", "MEET", "USER", "OFFER_COURSE", "IS_FRIEND_OF", "TAKE_COURSE", "MEET_AVAILABLE"]

# Read the SQL schema file
with open("schema.sql", 'r') as schema_file:
    schema = schema_file.read()

    # Split the schema into individual SQL statements
    statements = schema.split(';')

    # Execute each statement
    for statement in statements:
        statement = statement.strip()
        if statement:
            try:
                with engine.connect() as connection:
                    connection.execute(text(statement))
            except sqlite3.OperationalError as e:
                # Handle the error if the table already exists
                if "already exists" not in str(e):
                    print(f"Error: {e}")
                    print(f"Skipping statement: {statement}")
                    continue
for table in tables:
    print("Converting:", table)

# Read the CSV file
    df = pd.read_csv(f"{table}.csv")
    if table == "STUDY_GROUP" or table == "COURSE" or table == "JOIN_GROUP":
        pass
        del df[df.columns[0]]
    if table == "TAKE_COURSE":
        del df["sememter"]
        df["grade"] = 'I'
        df = df.drop_duplicates(subset=["course_ID", "user_ID"])
    if table == "DEPARTMENT":
        df = df.drop_duplicates(subset='department_ID')
    if table == "CONTACT":
        df = df.drop_duplicates(subset="user_ID")
    if table == "USER":
        df = df.drop_duplicates(subset="student_ID")
    if table == "JOIN_GROUP":
        df = df.drop_duplicates(subset=["group_ID", "user_ID"])
    if table == "MEET_AVAILABLE":
        df = df.drop_duplicates(
            subset=["meet_ID", "user_ID", "available_from", "available_to"])
    with engine.begin() as connection:
        df.to_sql(table, connection, if_exists='append', index=False)

# Commit the changes and close the connection


# "MEET_AVAILABLE_TIME", "TAKE_COURSE", "IS_FRIEND_OF","OFFER_COURSE", "USER"

OperationalError: (sqlite3.OperationalError) table DEPARTMENT already exists
[SQL: -- Table 3: DEPARTMENT
CREATE TABLE DEPARTMENT (
    department_ID CHAR(20) PRIMARY KEY NOT NULL,
    department_name VARCHAR(20) NOT NULL
)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)